In [1]:
import pandas as pd
from pathlib import Path
from scipy import stats
import json
import dabest
import pingouin as pg

In [2]:
def stat_test(data1, data2):
    _, p1 = stats.shapiro(data1)
    _, p2 = stats.shapiro(data2)
    
    val = pg.wilcoxon(data1, data2, alternative='two-sided')
    p = val['p-val'].values[0]
    eff_size = val['RBC'].values[0]
    eff_desc = "small" if abs(eff_size) < 0.3 else "medium" if abs(eff_size) < 0.5 else "large"
        
    p_value_threshold = 0.05
    
    if p < p_value_threshold:
        print(">>> Significant difference, p-value:", p)
        print(">>> Effect size:", eff_size)
        if p < 0.01:
            print(">>> Significant difference at 0.01")
        if p < 0.001:
            print(">>> Significant difference at 0.001")
    else:
        print("No significant difference")
        
    eff_size = eff_size if p < p_value_threshold else None
    eff_desc = eff_desc if p < p_value_threshold else None
    
    return p1, p2, p, eff_size, eff_desc

In [3]:
result_path = Path.cwd().parent / "analysis_results" / "summary.csv"
breakdown_data_path = Path.cwd().parent / "analysis_results" / "breakdown_data.json"

In [4]:
df = pd.read_csv(result_path, index_col=(0, 1, 2))
df.head()

model          task pe_technique  \
pythia-160m math-number  zero-shot  pythia-160m   math-number    zero-shot   
                         null-shot  pythia-160m   math-number    null-shot   
            math-algebra zero-shot  pythia-160m  math-algebra    zero-shot   
                         null-shot  pythia-160m  math-algebra    null-shot   
            anli         zero-shot  pythia-160m          anli    zero-shot   

                                    correct  total  accuracy  \
pythia-160m math-number  zero-shot        0    540       0.0   
                         null-shot        0    540       0.0   
            math-algebra zero-shot        0   1187       0.0   
                         null-shot        0   1187       0.0   
            anli         zero-shot       90   1200       7.5   

                                             created_at   mean_length  \
pythia-160m math-number  zero-shot  2024-05-04-08-53-45  10366.170370   
                         null-shot  2024-05-04-11-52-37  11041.246296   
            math-algebra zero-shot  2024-05-03-09-25-22  10021.422072   
                         null-shot  2024-05-03-16-00-41  10236.326032   
            anli         zero-shot  2024-03-02-10-13-32  13467.106667   

                                    median_length   std_length  min_length  \
pythia-160m math-number  zero-shot          10000  2502.600344        4331   
                         null-shot          11183  2716.999606        5558   
            math-algebra zero-shot           9558  2816.592556        3172   
                         null-shot           9825  2907.029382        4342   
            anli         zero-shot          13640  2102.988893         212   

                                    max_length  mean_time_taken  \
pythia-160m math-number  zero-shot       17701        19.871409   
                         null-shot       16882        19.775824   
            math-algebra zero-shot       27789        19.980845   
                         null-shot       23504        19.751473   
            anli         zero-shot       21359        32.997478   

                                    median_time_taken  std_time_taken  \
pythia-160m math-number  zero-shot          19.951003        0.506602   
                         null-shot          19.806719        0.174244   
            math-algebra zero-shot          19.958036        1.788766   
                         null-shot          19.790684        0.319809   
            anli         zero-shot          33.507348        2.462416   

                                    min_time_taken  max_time_taken  \
pythia-160m math-number  zero-shot       14.162299       20.882381   
                         null-shot       18.703898       20.600824   
            math-algebra zero-shot       14.680008       77.402771   
                         null-shot       15.446048       20.999895   
            anli         zero-shot        0.398390       34.808553   

                                    total_time_taken  
pythia-160m math-number  zero-shot      10730.560791  
                         null-shot      10678.945112  
            math-algebra zero-shot      23717.263572  
                         null-shot      23444.998943  
            anli         zero-shot      39596.973934

In [5]:
main_df = df[~df.index.get_level_values(0).str.contains("pythia|qwen|llama")]
main_df.head()

model          task  \
gemini-pro-chat math-number  zero-shot      gemini-pro-chat   math-number   
                             null-shot-cot  gemini-pro-chat   math-number   
                             zero-shot-cot  gemini-pro-chat   math-number   
                             null-shot      gemini-pro-chat   math-number   
                math-algebra zero-shot      gemini-pro-chat  math-algebra   

                                             pe_technique  correct  total  \
gemini-pro-chat math-number  zero-shot          zero-shot       87    540   
                             null-shot-cot  null-shot-cot      102    540   
                             zero-shot-cot  zero-shot-cot       90    540   
                             null-shot          null-shot       87    540   
                math-algebra zero-shot          zero-shot      295   1187   

                                             accuracy           created_at  \
gemini-pro-chat math-number  zero-shot      16.111111  2024-04-30-11-33-56   
                             null-shot-cot  18.888889  2024-04-30-12-20-47   
                             zero-shot-cot  16.666667  2024-04-30-11-59-20   
                             null-shot      16.111111  2024-04-30-11-46-18   
                math-algebra zero-shot      24.852570  2024-04-30-06-13-02   

                                            mean_length  median_length  \
gemini-pro-chat math-number  zero-shot        33.053704             14   
                             null-shot-cot   480.907407            329   
                             zero-shot-cot   199.811111             15   
                             null-shot        41.933333             14   
                math-algebra zero-shot        40.142376             14   

                                            std_length  min_length  \
gemini-pro-chat math-number  zero-shot      183.172320           9   
                             null-shot-cot  625.793667           0   
                             zero-shot-cot  512.343358           0   
                             null-shot      218.417878          13   
                math-algebra zero-shot      131.776185           9   

                                            max_length  mean_time_taken  \
gemini-pro-chat math-number  zero-shot            2054         1.123837   
                             null-shot-cot        5980         4.188187   
                             zero-shot-cot        5970         2.215503   
                             null-shot            2924         1.226178   
                math-algebra zero-shot            3585         1.070296   

                                            median_time_taken  std_time_taken  \
gemini-pro-chat math-number  zero-shot               0.862738        2.492253   
                             null-shot-cot           2.892331        5.092414   
                             zero-shot-cot           0.928071        3.917813   
                             null-shot               0.888910        2.764307   
                math-algebra zero-shot               0.868555        1.044572   

                                            min_time_taken  max_time_taken  \
gemini-pro-chat math-number  zero-shot            0.740963       29.687569   
                             null-shot-cot        0.805594       30.439521   
                             zero-shot-cot        0.749846       30.596659   
                             null-shot            0.775016       29.742396   
                math-algebra zero-shot            0.734635       29.855753   

                                            total_time_taken  
gemini-pro-chat math-number  zero-shot            606.871817  
                             null-shot-cot       2261.621232  
                             zero-shot-cot       1196.371361  
                             null-shot            662.136272  
                math-algebra zero-shot           1270.441327

# Overall comparison

In [6]:
zero_shot = main_df[main_df['pe_technique'] == 'zero-shot']['accuracy']
null_shot = main_df[main_df['pe_technique'] == 'null-shot']['accuracy']

print("Overall")
print(stat_test(zero_shot, null_shot))
print('---')

zero_shot_cot = main_df[main_df['pe_technique'] == 'zero-shot-cot']['accuracy']
null_shot_cot = main_df[main_df['pe_technique'] == 'null-shot-cot']['accuracy']

print("CoT Overall")
print(stat_test(zero_shot_cot, null_shot_cot))
print('---')

Overall
No significant difference
(1.60969150897472e-07, 4.131803507334553e-06, 0.17704197844605585, None, None)
---
CoT Overall
>>> Significant difference, p-value: 0.0031985852513144496
>>> Effect size: 0.2794304371485579
>>> Significant difference at 0.01
(6.504447810584679e-05, 1.0607135664031375e-05, 0.0031985852513144496, 0.2794304371485579, 'small')
---


# Task-wise comparison

In [7]:
for task in main_df['task'].unique():
    zero_shot = main_df[(main_df['task'] == task) & (main_df['pe_technique'] == 'zero-shot')]['accuracy']
    null_shot = main_df[(main_df['task'] == task) & (main_df['pe_technique'] == 'null-shot')]['accuracy']
    
    print(f"Task: {task}")
    print(stat_test(zero_shot, null_shot))
    print('---')
    
    zero_shot_cot = main_df[(main_df['task'] == task) & (main_df['pe_technique'] == 'zero-shot-cot')]['accuracy']
    null_shot_cot = main_df[(main_df['task'] == task) & (main_df['pe_technique'] == 'null-shot-cot')]['accuracy']
    
    if len(zero_shot_cot) > 0 and len(null_shot_cot) > 0:
        print(f"CoT Task: {task}")
        print(stat_test(zero_shot_cot, null_shot_cot))
        print('---')

Task: math-number
No significant difference
(0.2781377136707306, 0.20945793390274048, 1.0, None, None)
---
CoT Task: math-number
No significant difference
(0.4087183475494385, 0.22742445766925812, 0.375, None, None)
---
Task: math-algebra
No significant difference
(0.8827275633811951, 0.6989490985870361, 0.625, None, None)
---
CoT Task: math-algebra
No significant difference
(0.9025053381919861, 0.3649294972419739, 0.76953125, None, None)
---
Task: anli
No significant difference
(0.6353700757026672, 0.8392505645751953, 0.6953125, None, None)
---
CoT Task: anli
No significant difference
(0.9380416870117188, 0.5026674866676331, 0.16015625, None, None)
---
Task: triviaqa
>>> Significant difference, p-value: 0.037109375
>>> Effect size: 0.7454545454545454
(0.511067271232605, 0.03577284887433052, 0.037109375, 0.7454545454545454, 'large')
---
CoT Task: triviaqa
>>> Significant difference, p-value: 0.00390625
>>> Effect size: 0.9636363636363636
>>> Significant difference at 0.01
(0.5159384608

/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3351: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3351: UserWarning: Sample size too small for norma

# Model-wise comparison

In [8]:
for model in main_df['model'].unique():
    zero_shot = main_df[(main_df['model'] == model) & (main_df['pe_technique'] == 'zero-shot')]['accuracy']
    null_shot = main_df[(main_df['model'] == model) & (main_df['pe_technique'] == 'null-shot')]['accuracy']
    
    print(f"Model: {model}")
    print(stat_test(zero_shot, null_shot))
    print('---')
    
    zero_shot_cot = main_df[(main_df['model'] == model) & (main_df['pe_technique'] == 'zero-shot-cot')]['accuracy']
    null_shot_cot = main_df[(main_df['model'] == model) & (main_df['pe_technique'] == 'null-shot-cot')]['accuracy']
    
    if len(zero_shot_cot) > 0 and len(null_shot_cot) > 0:
        print(f"CoT Model: {model}")
        print(stat_test(zero_shot_cot, null_shot_cot))
        print('---')

Model: gemini-pro-chat
No significant difference
(0.017613448202610016, 0.04594356566667557, 0.2959265995411936, None, None)
---
CoT Model: gemini-pro-chat
No significant difference
(0.12673737108707428, 0.2297903150320053, 0.67877197265625, None, None)
---
Model: palm-2-chat
>>> Significant difference, p-value: 7.62939453125e-06
>>> Effect size: -0.9894736842105263
>>> Significant difference at 0.01
>>> Significant difference at 0.001
(0.0005348818958736956, 0.0012115583522245288, 7.62939453125e-06, -0.9894736842105263, 'large')
---
CoT Model: palm-2-chat
No significant difference
(0.0024478016421198845, 0.0030566509813070297, 0.389404296875, None, None)
---
Model: claude-3-opus
>>> Significant difference, p-value: 0.005329132080078125
>>> Effect size: 0.7052631578947368
>>> Significant difference at 0.01
(0.01336666289716959, 0.06693920493125916, 0.005329132080078125, 0.7052631578947368, 'large')
---
CoT Model: claude-3-opus
No significant difference
(0.5264863967895508, 0.1794133633

/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.p

# Breakdown comparison

## Data Preparation

In [9]:
raw_result_path = Path.cwd().parent / "results"

In [10]:
breakdown_data = {}
for model_path in raw_result_path.iterdir():
    if not model_path.is_dir():
        continue
    model = model_path.name
    breakdown_data[model] = {}
    
    for task_path in model_path.iterdir():
        if not task_path.is_dir():
            continue
        task = task_path.name
        breakdown_data[model][task] = {}
        
        for pe_path in task_path.iterdir():
            if not pe_path.is_dir():
                continue
            pe_technique = pe_path.name
            breakdown_data[model][task][pe_technique] = []
            
            for result_path in pe_path.iterdir():
                if not result_path.is_file() or result_path.suffix != ".json" or result_path.stem == "summary":
                    continue
                with result_path.open() as f:
                    result = json.load(f)
                    
                breakdown_data[model][task][pe_technique].append(1 if result["is_correct"] else 0)

In [11]:
with breakdown_data_path.open("w") as f:
    json.dump(breakdown_data, f)

## Analysis

In [12]:
with breakdown_data_path.open() as f:
    breakdown_data = json.load(f)

In [13]:
val = pg.wilcoxon(breakdown_data['gpt-3.5-turbo']['gsm8k']['zero-shot'], breakdown_data['gpt-3.5-turbo']['gsm8k']['null-shot'], alternative='two-sided')
val

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,25393.0,two-sided,1.535776e-08,-0.291005,0.458302


In [14]:
val['p-val'].values[0]

1.535776458045913e-08

In [15]:
dabest.effsize.cohens_h(breakdown_data['gpt-3.5-turbo']['gsm8k']['zero-shot'], breakdown_data['gpt-3.5-turbo']['gsm8k']['null-shot'])

0.16982534439935937

In [16]:
breakdown_result_csv = {
    "model": [],
    "task": [],
    "pe_technique": [],
    "significance at 0.05": [],
    "significance at 0.01": [],
    "significance at 0.001": [],
    "eff_size": [],
    "eff_desc": [],
    "is_improved": []
}
for model in breakdown_data:
    if model.startswith("pythia") or model.startswith("qwen") or model.startswith("llama"):
        continue
    for task in breakdown_data[model]:
        print(f"Model: {model}, Task: {task}")
        zero_shot = breakdown_data[model][task]['zero-shot']
        null_shot = breakdown_data[model][task]['null-shot']
        _, _, p, eff_size, eff_desc = stat_test(zero_shot, null_shot)
        is_improved = sum(breakdown_data[model][task]['null-shot']) > sum(breakdown_data[model][task]['zero-shot'])
        print('---')
        breakdown_result_csv["model"].append(model)
        breakdown_result_csv["task"].append(task)
        breakdown_result_csv["pe_technique"].append("zero-shot vs null-shot")
        breakdown_result_csv["significance at 0.05"].append(p < 0.05)
        breakdown_result_csv["significance at 0.01"].append(p < 0.01)
        breakdown_result_csv["significance at 0.001"].append(p < 0.001)
        breakdown_result_csv["eff_size"].append(eff_size)
        breakdown_result_csv["eff_desc"].append(eff_desc if eff_desc is not None else "")
        breakdown_result_csv["is_improved"].append(is_improved)
        
        if 'zero-shot-cot' in breakdown_data[model][task] and 'null-shot-cot' in breakdown_data[model][task]:
            zero_shot_cot = breakdown_data[model][task]['zero-shot-cot']
            null_shot_cot = breakdown_data[model][task]['null-shot-cot']
            print(f"CoT Model: {model}, Task: {task}")
            _, _, p, eff_size, eff_desc = stat_test(zero_shot_cot, null_shot_cot)
            is_improved = sum(breakdown_data[model][task]['null-shot-cot']) > sum(breakdown_data[model][task]['zero-shot-cot'])
            print('---')
            breakdown_result_csv["model"].append(model)
            breakdown_result_csv["task"].append(task)
            breakdown_result_csv["pe_technique"].append("zero-shot-cot vs null-shot-cot")
            breakdown_result_csv["significance at 0.05"].append(p < 0.05)
            breakdown_result_csv["significance at 0.01"].append(p < 0.01)
            breakdown_result_csv["significance at 0.001"].append(p < 0.001)
            breakdown_result_csv["eff_size"].append(eff_size)
            breakdown_result_csv["eff_desc"].append(eff_desc if eff_desc is not None else "")
            breakdown_result_csv["is_improved"].append(is_improved)

Model: gemini-pro-chat, Task: math-number
No significant difference
---
CoT Model: gemini-pro-chat, Task: math-number
No significant difference
---
Model: gemini-pro-chat, Task: math-algebra
>>> Significant difference, p-value: 0.014084670857345076
>>> Effect size: -0.23214285714285715
---
CoT Model: gemini-pro-chat, Task: math-algebra
>>> Significant difference, p-value: 3.994668363690491e-12
>>> Effect size: -0.4222222222222223
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: gemini-pro-chat, Task: anli
No significant difference
---
CoT Model: gemini-pro-chat, Task: anli
>>> Significant difference, p-value: 0.00011845696117386572
>>> Effect size: 0.3154362416107383
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: gemini-pro-chat, Task: triviaqa
>>> Significant difference, p-value: 1.6127895061806714e-95
>>> Effect size: 0.9534883720930232
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Mo

/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


No significant difference
---
Model: gemini-pro-chat, Task: race-m
No significant difference
---
CoT Model: gemini-pro-chat, Task: race-m
>>> Significant difference, p-value: 0.0022695643944767733
>>> Effect size: 0.3333333333333333
>>> Significant difference at 0.01
---
Model: gemini-pro-chat, Task: strategyqa
>>> Significant difference, p-value: 3.288406215761169e-58
>>> Effect size: 0.6582914572864322
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Model: gemini-pro-chat, Task: strategyqa
>>> Significant difference, p-value: 0.0
>>> Effect size: 0.9986657771847898
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: gemini-pro-chat, Task: winogrande
No significant difference
---
CoT Model: gemini-pro-chat, Task: winogrande
>>> Significant difference, p-value: 2.198892258939671e-173
>>> Effect size: 1.0
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: gemini-pro-chat, Task: math-pre-algebra
>

/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


>>> Significant difference, p-value: 5.144142228741655e-08
>>> Effect size: -0.3333333333333333
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: gemini-pro-chat, Task: race-h
>>> Significant difference, p-value: 0.0005389059834773553
>>> Effect size: -0.26627218934911245
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Model: gemini-pro-chat, Task: race-h
No significant difference
---
Model: gemini-pro-chat, Task: halueval-general
No significant difference
---
Model: gemini-pro-chat, Task: halueval-dialogue


...

>>> Significant difference, p-value: 0.0004546709769085569
>>> Effect size: -0.06791744840525332
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: race-m
>>> Significant difference, p-value: 8.426314328192002e-23
>>> Effect size: 0.4830917874396135
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Model: claude-3-sonnet, Task: race-m
>>> Significant difference, p-value: 1.1396626079862228e-09
>>> Effect size: -0.2808510638297872
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: strategyqa
>>> Significant difference, p-value: 9.293496288971375e-115
>>> Effect size: 0.6803571428571429
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Model: claude-3-sonnet, Task: strategyqa
>>> Significant difference, p-value: 4.382429096537627e-202
>>> Effect size: 0.9055258467023173
>>> Significant difference at 0.01
>>> Significant diff

/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


>>> Significant difference, p-value: 7.868655909081431e-47
>>> Effect size: 0.214016411621202
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: race-h
>>> Significant difference, p-value: 1.3319362149167647e-37
>>> Effect size: 0.42116630669546434
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
CoT Model: claude-3-sonnet, Task: race-h
>>> Significant difference, p-value: 2.1271033670012279e-13
>>> Effect size: -0.20181405895691612
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: halueval-general
No significant difference
---
Model: claude-3-sonnet, Task: halueval-dialogue


/Users/pittawat/dev/miniconda3/envs/null-shot-prompting/lib/python3.11/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


>>> Significant difference, p-value: 1.1522324765694896e-19
>>> Effect size: -0.2400279916025192
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: math-geometry
>>> Significant difference, p-value: 0.015442095157822175
>>> Effect size: 0.29411764705882354
---
CoT Model: claude-3-sonnet, Task: math-geometry
>>> Significant difference, p-value: 8.88607392654154e-23
>>> Effect size: 0.8394160583941606
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: math-count-prob
No significant difference
---
CoT Model: claude-3-sonnet, Task: math-count-prob
>>> Significant difference, p-value: 2.1663622326265138e-27
>>> Effect size: 0.8709677419354838
>>> Significant difference at 0.01
>>> Significant difference at 0.001
---
Model: claude-3-sonnet, Task: math-pre-calc
No significant difference
---
CoT Model: claude-3-sonnet, Task: math-pre-calc
>>> Significant difference, p-value: 2.2975625870

In [17]:
breakdown_result_df = pd.DataFrame(breakdown_result_csv)
breakdown_result_df.to_csv(Path.cwd().parent / "analysis_results" / "breakdown_result.csv")

In [18]:
breakdown_result_df.head()

,model,task,pe_technique,significance at 0.05,significance at 0.01,significance at 0.001,eff_size,eff_desc,is_improved
0,gemini-pro-chat,math-number,zero-shot vs null-shot,False,False,False,NaN,,False
1,gemini-pro-chat,math-number,zero-shot-cot vs null-shot-cot,False,False,False,NaN,,True
2,gemini-pro-chat,math-algebra,zero-shot vs null-shot,True,False,False,-0.232143,small,True
3,gemini-pro-chat,math-algebra,zero-shot-cot vs null-shot-cot,True,True,True,-0.422222,medium,True
4,gemini-pro-chat,anli,zero-shot vs null-shot,False,False,False,NaN,,True


In [19]:
breakdown_result_df = pd.read_csv(Path.cwd().parent / "analysis_results" / "breakdown_result.csv", index_col=0)

In [20]:
breakdown_result_df.head()

,model,task,pe_technique,significance at 0.05,significance at 0.01,significance at 0.001,eff_size,eff_desc,is_improved
0,gemini-pro-chat,math-number,zero-shot vs null-shot,False,False,False,NaN,NaN,False
1,gemini-pro-chat,math-number,zero-shot-cot vs null-shot-cot,False,False,False,NaN,NaN,True
2,gemini-pro-chat,math-algebra,zero-shot vs null-shot,True,False,False,-0.232143,small,True
3,gemini-pro-chat,math-algebra,zero-shot-cot vs null-shot-cot,True,True,True,-0.422222,medium,True
4,gemini-pro-chat,anli,zero-shot vs null-shot,False,False,False,NaN,NaN,True


In [21]:
breakdown_result_df.pe_technique.unique()

array(['zero-shot vs null-shot', 'zero-shot-cot vs null-shot-cot'],
      dtype=object)

In [22]:
main_tasks = ['aqua', 'gsm8k', 'strategyqa', 'winogrande', 'race-m', 'race-h', 'anli', 'triviaqa']
math_tasks = ['math-number', 'math-algebra', 'math-pre-algebra', 'math-int-algebra', 'math-geometry', 'math-count-prob', 'math-pre-calc']
halueval_tasks = ['halueval-general', 'halueval-dialogue', 'halueval-summarization', 'halueval-qa']

stat_sig_cols = ['significance at 0.05', 'significance at 0.01', 'significance at 0.001']

In [23]:
def get_significant_count(selected_tasks, prompt):
    bk_result_df = breakdown_result_df[breakdown_result_df['task'].isin(selected_tasks)]
    bk_result_df = bk_result_df[bk_result_df['pe_technique'] == prompt]
    print("Number of combinations: ", len(bk_result_df))
    
    improved_bk_result_df = bk_result_df[bk_result_df['is_improved']]
    bk_result_df = improved_bk_result_df[improved_bk_result_df['task'].isin(selected_tasks)]
    print("Number of improved combinations: ", len(bk_result_df))
    
    sig_bk_result_df = bk_result_df[bk_result_df[stat_sig_cols].any(axis=1)]
    print("Number of improved and significant combinations: ", len(sig_bk_result_df))
    
    eff_desc = sig_bk_result_df['eff_desc']
    print("Effect size distribution:")
    print(eff_desc.value_counts())

In [24]:
get_significant_count(main_tasks, 'zero-shot vs null-shot')

Number of combinations:  80
Number of improved combinations:  34
Number of improved and significant combinations:  20
Effect size distribution:
medium    12
small      7
large      1
Name: eff_desc, dtype: int64


In [25]:
get_significant_count(math_tasks, 'zero-shot vs null-shot')

Number of combinations:  70
Number of improved combinations:  33
Number of improved and significant combinations:  17
Effect size distribution:
medium    7
small     5
large     5
Name: eff_desc, dtype: int64


In [26]:
get_significant_count(halueval_tasks, 'zero-shot vs null-shot')

Number of combinations:  40
Number of improved combinations:  25
Number of improved and significant combinations:  14
Effect size distribution:
medium    6
small     6
large     2
Name: eff_desc, dtype: int64


In [27]:
get_significant_count(main_tasks, 'zero-shot-cot vs null-shot-cot')

Number of combinations:  80
Number of improved combinations:  19
Number of improved and significant combinations:  11
Effect size distribution:
medium    4
small     4
large     3
Name: eff_desc, dtype: int64


In [28]:
get_significant_count(math_tasks, 'zero-shot-cot vs null-shot-cot')

Number of combinations:  70
Number of improved combinations:  40
Number of improved and significant combinations:  10
Effect size distribution:
medium    7
small     3
Name: eff_desc, dtype: int64
